# Modelling: Random Forest Berlin rent

Classification with imbalanced data using random forest.
Comparison of accuracy score using downsampling and SMOTE upsampling

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
## multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# read the clean data without outliers and only numeric (with dummies)
df = pd.read_csv("df_numeric.csv")
df

,total rent,area,number of rooms,balcony,built-in kitchen,basement,garden,elevator,stepless,guest toilet,flat share possible,district_alt-hohenschönhausen,district_alt-treptow,district_altglienicke,district_baumschulenweg,district_biesdorf,district_blankenburg,district_bohnsdorf,district_borsigwalde,district_britz,district_buch,district_buckow,district_charlottenburg,district_charlottenburg-nord,district_dahlem,district_falkenberg,district_falkenhagener feld,district_fennpfuhl,district_französisch buchholz,district_friedenau,district_friedrichsfelde,district_friedrichshagen,district_friedrichshain,district_gatow,district_gesundbrunnen,district_gropiusstadt,district_grunewald,district_grünau,district_hakenfelde,district_halensee,district_hansaviertel,district_haselhorst,district_heiligensee,district_heinersdorf,district_hellersdorf,district_hermsdorf,district_johannisthal,district_karlshorst,district_karow,district_kaulsdorf,district_konradshöhe,district_kreuzberg,district_köpenick,district_lankwitz,district_lichtenberg,district_lichtenrade,district_lichterfelde,district_lübars,district_mahlsdorf,district_mariendorf,district_marienfelde,district_marzahn,district_mitte,district_moabit,district_märkisches viertel,district_müggelheim,district_neu-hohenschönhausen,district_neukölln,district_niederschöneweide,district_niederschönhausen,district_nikolassee,district_oberschöneweide,district_pankow,district_plänterwald,district_prenzlauer berg,district_rahnsdorf,district_reinickendorf,district_rosenthal,district_rummelsburg,district_schmargendorf,district_schmöckwitz,district_schöneberg,district_siemensstadt,district_spandau,district_staaken,district_steglitz,district_tegel,district_tempelhof,district_tiergarten,district_wannsee,district_wedding,district_weißensee,district_westend,district_wilhelmsruh,district_wilhelmstadt,district_wilmersdorf,district_wittenau,district_zehlendorf,landlord_degewo,landlord_estate agent,landlord_housinganywhere b.v.,landlord_howoge,landlord_numa group,landlord_private offer,landlord_tauschwohnung wohnungstausch,landlord_visionapartments,landlord_wohnungsswap.de
0,3940.00,175.00,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,4030.00,147.00,4.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1404.41,72.86,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2663.13,107.89,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2790.64,93.36,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,1035.00,80.00,3.0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3331,1663.00,84.10,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Dropping columns with low number of occurences

In [3]:
df = df.drop(columns=["guest toilet", "flat share possible", "stepless", "garden"])

#### Doing X-y split (y is the target variable total rent)

In [4]:
y = df["total rent"]

In [5]:
X = df.drop(columns=["total rent"])
X

,area,number of rooms,balcony,built-in kitchen,basement,elevator,district_alt-hohenschönhausen,district_alt-treptow,district_altglienicke,district_baumschulenweg,district_biesdorf,district_blankenburg,district_bohnsdorf,district_borsigwalde,district_britz,district_buch,district_buckow,district_charlottenburg,district_charlottenburg-nord,district_dahlem,district_falkenberg,district_falkenhagener feld,district_fennpfuhl,district_französisch buchholz,district_friedenau,district_friedrichsfelde,district_friedrichshagen,district_friedrichshain,district_gatow,district_gesundbrunnen,district_gropiusstadt,district_grunewald,district_grünau,district_hakenfelde,district_halensee,district_hansaviertel,district_haselhorst,district_heiligensee,district_heinersdorf,district_hellersdorf,district_hermsdorf,district_johannisthal,district_karlshorst,district_karow,district_kaulsdorf,district_konradshöhe,district_kreuzberg,district_köpenick,district_lankwitz,district_lichtenberg,district_lichtenrade,district_lichterfelde,district_lübars,district_mahlsdorf,district_mariendorf,district_marienfelde,district_marzahn,district_mitte,district_moabit,district_märkisches viertel,district_müggelheim,district_neu-hohenschönhausen,district_neukölln,district_niederschöneweide,district_niederschönhausen,district_nikolassee,district_oberschöneweide,district_pankow,district_plänterwald,district_prenzlauer berg,district_rahnsdorf,district_reinickendorf,district_rosenthal,district_rummelsburg,district_schmargendorf,district_schmöckwitz,district_schöneberg,district_siemensstadt,district_spandau,district_staaken,district_steglitz,district_tegel,district_tempelhof,district_tiergarten,district_wannsee,district_wedding,district_weißensee,district_westend,district_wilhelmsruh,district_wilhelmstadt,district_wilmersdorf,district_wittenau,district_zehlendorf,landlord_degewo,landlord_estate agent,landlord_housinganywhere b.v.,landlord_howoge,landlord_numa group,landlord_private offer,landlord_tauschwohnung wohnungstausch,landlord_visionapartments,landlord_wohnungsswap.de
0,175.00,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,147.00,4.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,72.86,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,107.89,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,93.36,3.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,80.00,3.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3331,84.10,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3332,72.00,2.5,0,0,1,0,0,0,0,0,0,0

#### Train/ Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,random_state= 42)

#### Standardizing the data using a scaler is not necessary but can be done to improve model (in this case model stays the same)

In [7]:
# fit and transform for tain data
# only transform for test data
#transformer = StandardScaler()
#X_train = transformer.fit_transform(X_train)
#X_test  = transformer.transform(X_test)

#### Since SMOTE works on numerical data only, we encode (dummify) the categorical variables, however SMOTE is best to use for categorical target variables, which is why we didn't use it in the end

#### No up- or downsampling since dataset is balanced enough to predict rent

In [8]:
#def down_samp_rand(Xin, yin, ratio=1):
        #from imblearn.under_sampling import RandomUnderSampler
        #"""Downsamples majority class using random sampling.
       # Ratio argument is the ratio of minority class to the downsampled majority
       # """
        #rus = RandomUnderSampler(sampling_strategy=ratio, random_state=RAND_STATE)
        #X_rus, y_rus = rus.fit_resample(Xin, yin)
        #return X_rus, y_rus

In [9]:
#X_train, y_train = down_samp_rand(X_train,y_train)

In [10]:
# check that we have downsampled
#y_train.value_counts()

In [11]:
# check for missing values
X_test.isna().any(axis=1).value_counts()
X_train.isna().any(axis=1).value_counts()

False    667
dtype: int64

False    2668
dtype: int64

In [12]:
# remove the null values
na_idx = X_train[X_train.isna().any(axis=1)].index
X_train = pd.DataFrame(X_train).drop(na_idx)
y_train = pd.DataFrame(y_train).drop(na_idx)


#### Train, fit, and evaluate random forest regression model

In [13]:
from sklearn.ensemble import RandomForestRegressor
rf_ops = {"max_depth":6,
           "min_samples_leaf":20,
           "max_features":None,
           "n_estimators":100,
           "bootstrap":True,
           "oob_score":True,
           "random_state":42}
regressor = RandomForestRegressor(**rf_ops)
regressor.fit(X_train, y_train)
print("train prediction R2 score: %.2f" %(regressor.score(X_train, y_train)))
print("test prediction R2 score: %.2f"  %(regressor.score(X_test, y_test)))


RandomForestRegressor(max_depth=6, max_features=None, min_samples_leaf=20,
                      oob_score=True, random_state=42)

train prediction R2 score: 0.55
test prediction R2 score: 0.47


#### Cross validation

In [14]:
from sklearn.model_selection import cross_val_score
folds=5
cross_val_scores = cross_val_score(regressor, X_train, y_train, cv=folds)


In [15]:
print("cv scores over {:d} iterations: \n".format(folds))
cross_val_scores

cv scores over 5 iterations: 



array([0.53823603, 0.51262605, 0.48181102, 0.5324638 , 0.51437579])

In [16]:
# 5 folds, so the algorithm was trained and evaluated 5 times, and the resulting scores for each fold are given in the array
# these scores are measures of the model's performance on the validation set for each fold. 
# the array shows that the scores for the five folds were 0.45403997, 0.42341055, 0.38559813, 0.44685216, and 0.47643573, respectively
# by looking at these scores, you can get an idea of how well the model is performing on different parts of the data. 
# a large variation in the scores may indicate that the model is overfitting or that the data is not consistent. 
# a consistent, high score across all folds suggests that the model is performing well and is generalizing to new data.

In [17]:
print("the std. dev. in the cv scores is {:.4f}".format(np.std(cross_val_scores)))

the std. dev. in the cv scores is 0.0197


In [18]:
# when you calculate the standard deviation of the cross-validation scores using np.std, you get a single number 
# that represents the variation in the cross-validation scores across all the folds. 
# in your case, the result is 0.0309, which means that the cross-validation scores vary by approximately 0.0309 on average across all the folds.
# the standard deviation is a measure of the spread or variability of a set of values.  
# large standard deviation indicates that the values are more spread out or variable, 
# while a small standard deviation indicates that the values are more tightly clustered around the mean.

# in the context of cross-validation, a small standard deviation indicates that the model is performing consistently 
# across all the folds, while a large standard deviation indicates that the model's performance is more variable across the folds. 
# Ideally, you want the standard deviation to be as small as possible, 
# since this indicates that the model is stable and reliable across different parts of the data.

# in your case, a standard deviation of 0.0309 suggests that the model's performance is relatively consistent across the folds,
# which is a good sign. 